In [24]:
import pandas as pd
import numpy as np
from calendar import monthrange, isleap
from collections import defaultdict
import datetime
from openpyxl import Workbook
import os
from shutil import copyfile

In [25]:
revenue_target_path = 'REPORTED REVENUE_TARGET.xlsx'
revenue_actual_path = 'Revenue(exc.USO) 202209.xlsx'
template_path = 'Data_Input_20210318.xlsx'
OUTPUT_DIR = 'output/'

year = '2022'
ALL_MONTHS = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
# OUTPUT_COLS = ['ORGID', 'ORGLEVEL', 'KPIID', 'ACTUAL', 'TM_KEY', 'CYCLE', 'DATA_AS_OF', 'NUMERATOR', 'DENOMINATOR']


In [26]:
# current_week_no = datetime.date.today().isocalendar()[1]
# current_month_no = int(datetime.date.today().strftime("%m"))

In [27]:
def get_month_in_week_number(year):
    month_in_week_number = []
    for week in range(0,54):
        input_date = f'{year}-W{week}'
        first_day_of_week = datetime.datetime.strptime(input_date + '-1', '%G-W%V-%u')
        last_day_of_week = first_day_of_week + datetime.timedelta(days=6)
        if first_day_of_week.month == last_day_of_week.month:
            month_days = [[first_day_of_week.month, 7]]
        else:
            month_days = [[first_day_of_week.month, monthrange(int(year), first_day_of_week.month)[1] - first_day_of_week.day + 1],
                  [last_day_of_week.month, last_day_of_week.day]]
        month_in_week_number.append(month_days)

    # remove Dec last year
    if len(month_in_week_number[0]) == 1 and month_in_week_number[0][0][0] == 12:
        month_in_week_number = month_in_week_number[1:]
        
    # remove Dec in Jan
    if month_in_week_number[0][0][0] == 12:
        month_in_week_number[0] = [month_in_week_number[0][1]]
        
    # remove next Jan
    if month_in_week_number[-1][0][0] == 1:
        month_in_week_number = month_in_week_number[:-1]
        
    # remove Next Jan in Dec
    if len(month_in_week_number[-1]) == 2:
        month_in_week_number[-1] = month_in_week_number[-1][:1]
    
    # merge week if not leap year and not 52
    if (not isleap(int(year))) and (len(month_in_week_number) == 53):
        month_in_week_number[1][0][1] += month_in_week_number[0][0][1]
        month_in_week_number = month_in_week_number[1:]
    
    return month_in_week_number

def _get_daily_target_by_month(month_target):
    day_in_month = [monthrange(int(year), month_no)[1] for month_no in range(1, 13)]
    return np.array(month_target) / day_in_month

def get_week_targets(month_target, year):
    month_in_week_number = get_month_in_week_number(year)
    daily_target_by_month = _get_daily_target_by_month(month_target)
    week_targets = []
    for row in month_in_week_number:
        week_target = 0
        for month_no, day_count in row:
            week_target += (daily_target_by_month[month_no-1] * day_count)
        week_targets.append(week_target)
    return week_targets

def get_ref_maps(ref_name):
    kpi_map, ref_map = {}, {}
    revenue_ref = refcode[refcode['KPI'] == ref_name]
    for i, row in revenue_ref.iterrows():
        kpi_map[row['Org ID']] = str(row['Data ID'])
        ref_map[row['Org ID']] = str(row['Ref Code'])
    return kpi_map, ref_map

def get_stamp_date(year):
    year_all = []
    month_all = []
    day_all = []
    for week_no in range(1,53):
        input_date = f'{year}-W{str(week_no)[-2:]}'
        stamp_date = datetime.datetime.strptime(input_date + '-1', '%G-W%V-%u') + datetime.timedelta(days=6)
        year_all.append(stamp_date.year)
        month_all.append(stamp_date.month)
        day_all.append(stamp_date.day)
    return year_all, month_all, day_all

target

In [28]:
revenue_target = pd.read_excel(revenue_target_path)
revenue_target = revenue_target[revenue_target['YEAR'] == int(year)].reset_index(drop=True)

In [29]:
targets = {}
for i, row in revenue_target.iterrows():
    month_targets = row[ALL_MONTHS]
    week_targets = get_week_targets(month_targets, year)

    accu_month_targets = np.cumsum(month_targets)
    accu_week_targets = np.cumsum(week_targets)
    
    targets[row['PRODUCT']] = {'week': accu_week_targets, 'month': accu_month_targets}

In [30]:
targets

{'REPORTED TOTAL REVENUE': {'week': array([2.30000916e+09, 4.08890517e+09, 5.87780118e+09, 7.66669719e+09,
         9.65221896e+09, 1.16705117e+10, 1.36888044e+10, 1.57070972e+10,
         1.75880600e+10, 1.94461345e+10, 2.13042091e+10, 2.31622836e+10,
         2.50624514e+10, 2.70187435e+10, 2.89750356e+10, 3.09313277e+10,
         3.28836221e+10, 3.48119303e+10, 3.67402385e+10, 3.86685468e+10,
         4.05968550e+10, 4.25973967e+10, 4.46268317e+10, 4.66562667e+10,
         4.86857018e+10, 5.07023740e+10, 5.27020293e+10, 5.47016846e+10,
         5.67013399e+10, 5.87009951e+10, 6.07364703e+10, 6.27719454e+10,
         6.48074206e+10, 6.68428957e+10, 6.89384077e+10, 7.10789473e+10,
         7.32194869e+10, 7.53600265e+10, 7.74911376e+10, 7.95986774e+10,
         8.17062172e+10, 8.38137570e+10, 8.59212968e+10, 8.81210134e+10,
         9.03360929e+10, 9.25511724e+10, 9.47662519e+10, 9.69611046e+10,
         9.91407873e+10, 1.01320470e+11, 1.03500152e+11, 1.05368452e+11]),
  'month': JAN 

Actual

In [31]:
revenue_actual = pd.read_excel(revenue_actual_path, sheet_name='22 Actual', header=None)
revenue_actual = revenue_actual[:65]

In [32]:
KEYWORDS = [' True consol', 'Postpaid', 'Prepaid', 'Broadband', 
            'Subscription & installation', 'Online', 'Pay TV',
            'New business', 'Mobile']

In [33]:
product_row = {}
for kw in KEYWORDS:
#     print(kw)
#     print(np.where(revenue_actual==kw)[0][0])
    product_row[kw] = np.where(revenue_actual==kw)[0][0]
    
month_cols = []
for month_no in range(1,13):
    date = datetime.datetime(int(year), month_no, 1)
#     print(np.where(revenue_actual==date)[1][0])
    month_cols.append(np.where(revenue_actual==date)[1][0])
    
code_map = {}
for i, row in revenue_target.iterrows():
    code_map[row['PRODUCT']] = row['KPIID']

month KPI

In [34]:
all_kpis_actual = {}
for kpi_name in product_row.keys():
    print('####', kpi_name)
    kpi_scores = [revenue_actual.iloc[product_row[kpi_name], month_cols[0]-1]]
    for month_no in range(1, 13):
        month_score = revenue_actual.iloc[product_row[kpi_name], month_cols[month_no-1]]
        if (month_score != month_score) or (month_score < 1):
            month_score = kpi_scores[-1]
        kpi_scores.append(month_score)
    kpi_scores = kpi_scores[1:]
#     print(kpi_name, kpi_scores)
    kpi_scores = [s*(10**6) for s in kpi_scores]
#     print(kpi_name, kpi_scores)
    all_kpis_actual[kpi_name] = kpi_scores

####  True consol
#### Postpaid
#### Prepaid
#### Broadband
#### Subscription & installation
#### Online
#### Pay TV
#### New business
#### Mobile


In [35]:
all_kpis_actual

{' True consol': [8349691650.683502,
  8161873708.652685,
  9119796107.454475,
  8216741261.073172,
  8359912274.690848,
  9142915799.453382,
  8289458848.2659,
  8336261525.914724,
  6977608874.63656,
  6977608874.63656,
  6977608874.63656,
  6977608874.63656],
 'Postpaid': [3660386215.261717,
  3643820323.7017145,
  3589291240.146403,
  3559781778.660469,
  3530103586.1025,
  3549616789.927799,
  3557001569.5144,
  3581010135.7495027,
  3587305886.82472,
  3587305886.82472,
  3587305886.82472,
  3587305886.82472],
 'Prepaid': [2033823436.4866664,
  1810788955.3799999,
  2077120047.7866669,
  1940381810.1796434,
  2021755202.1275,
  2043199271.8521996,
  2023585942.3356001,
  2035727989.6105099,
  1972431335.68184,
  1972431335.68184,
  1972431335.68184,
  1972431335.68184],
 'Broadband': [1486274131.7584467,
  1458585907.0200002,
  1470673203.2761567,
  1438078443.3353965,
  1434362562.4399998,
  1391259218.2300005,
  1409796792.45,
  1418922218.3400002,
  1417871652.13,
  1417871652

revenue

In [36]:
revenues = []
revenue_factors = ['Mobile', 'Online', 'Pay TV', 'New business']
for i in range(12):
    revenue = sum([all_kpis_actual[kpi][i] for kpi in revenue_factors])
    revenues.append(revenue)

all_kpis_actual['revenue'] = revenues

In [37]:
actual = {}
for kpi in all_kpis_actual.keys():
    month_targets = all_kpis_actual[kpi]
    week_targets = get_week_targets(month_targets, year)

    accu_month_targets = np.cumsum(month_targets)
    accu_week_targets = np.cumsum(week_targets)
    
    actual[kpi] = {'week': accu_week_targets, 'month': accu_month_targets}

In [38]:
actual

{' True consol': {'week': array([2.42410403e+09, 4.30951827e+09, 6.19493252e+09, 8.08034676e+09,
         1.00986646e+10, 1.21391330e+10, 1.41796014e+10, 1.62200699e+10,
         1.82766872e+10, 2.03359960e+10, 2.23953048e+10, 2.44546136e+10,
         2.64530356e+10, 2.83702752e+10, 3.02875148e+10, 3.22047545e+10,
         3.41177773e+10, 3.60054994e+10, 3.78932216e+10, 3.97809437e+10,
         4.16686658e+10, 4.37318343e+10, 4.58651813e+10, 4.79985283e+10,
         5.01318754e+10, 5.21531365e+10, 5.40249498e+10, 5.58967631e+10,
         5.77685764e+10, 5.96403897e+10, 6.15227713e+10, 6.34051529e+10,
         6.52875346e+10, 6.71699162e+10, 6.89069990e+10, 7.05351078e+10,
         7.21632165e+10, 7.37913252e+10, 7.54044284e+10, 7.69800175e+10,
         7.85556066e+10, 8.01311957e+10, 8.17067848e+10, 8.33273907e+10,
         8.49554994e+10, 8.65836082e+10, 8.82117169e+10, 8.98098144e+10,
         9.13854035e+10, 9.29609926e+10, 9.45365817e+10, 9.58870867e+10]),
  'month': array([8.34969

Write to Template

In [39]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [40]:
name_map = {'REPORTED TOTAL REVENUE': 'revenue',
         'REPORTED POSTPAID REVENUE': 'Postpaid',
         'REPORTED PREPAID REVENUE': 'Prepaid',
         'REPORTED BROADBAND REVENUE': 'Broadband',
         'REPORTED TVS REVENUE': 'Subscription & installation'}

ref_code = {'REPORTED TOTAL REVENUE': ['413129', 'SW4J'],
         'REPORTED POSTPAID REVENUE': ['413127', 'YY38'],
         'REPORTED PREPAID REVENUE': ['413128', 'U3V6'],
         'REPORTED BROADBAND REVENUE': ['413126', 'O2SD'],
         'REPORTED TVS REVENUE': ['413131', 'R9L4']}

In [41]:
year_all, month_all, day_all = get_stamp_date(year)

In [42]:
# kpi_name = 'REPORTED TOTAL REVENUE'

# template = pd.read_excel(template_path, sheet_name='data')

# template['Numerator'] = actual[name_map[kpi_name]]['week']
# template['Denominator'] = targets[kpi_name]['week']
# template['Value'] = template['Numerator'] / template['Denominator'] * 100

# template['Data_ID'] = ref_code[kpi_name][0]
# template['Ref_Code'] = ref_code[kpi_name][1]

# template['Year'] = year_all
# template['Month'] = month_all
# template['Date'] = day_all


# output_file_name = f'{OUTPUT_DIR}/{kpi_name}.xlsx'
# copyfile(template_path, output_file_name)

# writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')
# template.to_excel(writer, 'data', index=False)
# info = pd.read_excel(template_path, sheet_name='info')
# info.to_excel(writer, sheet_name = 'info', index=False)

# writer.save()
# writer.close()

# template

In [43]:
for kpi_name in targets.keys():
    template = pd.read_excel(template_path, sheet_name='data')

    template['Numerator'] = actual[name_map[kpi_name]]['week']
    template['Denominator'] = targets[kpi_name]['week']
    template['Value'] = template['Numerator'] / template['Denominator'] * 100

    template['Data_ID'] = ref_code[kpi_name][0]
    template['Ref_Code'] = ref_code[kpi_name][1]

    template['Year'] = year_all
    template['Month'] = month_all
    template['Date'] = day_all


    output_file_name = f'{OUTPUT_DIR}/{kpi_name}.xlsx'
    copyfile(template_path, output_file_name)

    writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')
    template.to_excel(writer, 'data', index=False)
    info = pd.read_excel(template_path, sheet_name='info')
    info.to_excel(writer, sheet_name = 'info', index=False)

    writer.save()
    writer.close()

Combine file

In [44]:
template = pd.DataFrame()
for kpi_name in targets.keys():
    output_file_name = f'{OUTPUT_DIR}/{kpi_name}.xlsx'
    data_temp = pd.read_excel(output_file_name, sheet_name='data')
    template = template.append(data_temp)

In [45]:
output_file_name = f'{OUTPUT_DIR}/all_reported_revenues.xlsx'
copyfile(template_path, output_file_name)

writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')
template.to_excel(writer, 'data', index=False)
info = pd.read_excel(template_path, sheet_name='info')
info.to_excel(writer, sheet_name = 'info', index=False)

writer.save()
writer.close()